# Fetch the data into a website(1Mg Homeopathic) by using Selenium and BeautifulSoup
  


## Table - 2 : medicine_details
`name` - Name of the medicine

`brand_name`  - Brand name

`key_benefits` - Key Benefits area (Hair, eye, joint, skin)

`key_ingredients` - Ingredient of medicine

`rating` - user rating of the medicine

`number_of_rating`  - Number of people rated that product

In [19]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Function to fetch additional information from a given URL
def fetch_additional_info(url):
    driver = webdriver.Chrome()
    driver.get(url)
    
    # Wait for some time to ensure the page is fully loaded
    driver.implicitly_wait(2)
    
    # Extract information using BeautifulSoup 
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Remove dropdown
    close_popup_button = driver.find_element(By.CLASS_NAME, 'UpdateCityModal__cancel-btn___2jWwS')
    close_popup_button.click()
    time.sleep(2)

    
    # Extracting the Name, Brand Name, Rating, Number of Rating, Key Benefits,  Key Ingredients
    name_of_medicine_element = soup.find('h1', {'class': 'ProductTitle__product-title___3QMYH'})
    name_of_medicine = name_of_medicine_element.text.strip() if name_of_medicine_element else 'NA'
    
    brand_name_element = soup.find('div', {'class': 'ProductTitle__marketer___7Wsj9'})
    brand_name = brand_name_element.text.strip() if brand_name_element else 'NA'
        
    rating_element = soup.find('div', {'class': 'RatingDisplay__ratings-container___3oUuo'})
    rating = rating_element.text.strip() if rating_element else 'NA'
    
    num_rating_element = soup.find('span', {'class': 'RatingDisplay__ratings-header___ZNj5b'})
    num_rating = num_rating_element.text.strip() if num_rating_element else 'NA'
    
  
   
    

    
    key_benefits_element = soup.find('div', {'class': 'ProductHighlights__product-highlights___2jAF5'})
    key_benefits_items = key_benefits_element.find_all('li') if key_benefits_element else []
    key_benefits_data = [item.text.strip() for item in key_benefits_items] if key_benefits_items else 'NA'
    if key_benefits_data == ['NA']:
        key_benefits = 'NA'
    else:
        key_benefits = '\n'.join(f'{index + 1}. {element}' for index, element in enumerate(key_benefits_data, start=0))
    
    # Close the WebDriver
    driver.quit()
    
    return name_of_medicine, brand_name, rating, num_rating, key_benefits




# Specify the path to your CSV files
csv_file_path_input = 'medicine_name_data_1200.csv'
csv_file_path_output = 'medicine_details.csv'

# Create a list to store all the 1Mg URLs
all_1mg_urls = []

# Read data from the CSV file and extract '1Mg_URL'
with open(csv_file_path_input, 'r', encoding='utf-8') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        url = row['1Mg_URL']
        all_1mg_urls.append(url)

# Create a list to store the fetched information
all_additional_info = []

# Iterate through each URL and fetch additional information
for url in all_1mg_urls:
    name, brand, rating, num_rating, key_benefits = fetch_additional_info(url)
    all_additional_info.append({
        'Name': name , 
        'Brand_Name': brand, 
        'Rating' : rating, 
        'Number_of_rating' : num_rating,
        'Key Benefits' :  key_benefits   
    })

# Create a new CSV file and write all the data
with open(csv_file_path_output, 'w', newline='', encoding='utf-8') as csvfile:
    # Create a CSV writer
    csv_writer = csv.writer(csvfile)

    # Write header row
    header = ['Name', 'Brand_Name', 'Rating', 'Number_of_rating', 'Key Benefits']
    csv_writer.writerow(header)

    # Write all the data to the CSV file
    for info in all_additional_info:
        csv_writer.writerow([
            info['Name'],
            info['Brand_Name'],
            info['Rating'],
            info['Number_of_rating'],
            
            info['Key Benefits']
        ])


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".UpdateCityModal__cancel-btn___2jWwS"}
  (Session info: chrome=120.0.6099.217); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E18D4D02+56194]
	(No symbol) [0x00007FF7E18404B2]
	(No symbol) [0x00007FF7E16E76AA]
	(No symbol) [0x00007FF7E17316D0]
	(No symbol) [0x00007FF7E17317EC]
	(No symbol) [0x00007FF7E1774D77]
	(No symbol) [0x00007FF7E1755EBF]
	(No symbol) [0x00007FF7E1772786]
	(No symbol) [0x00007FF7E1755C23]
	(No symbol) [0x00007FF7E1724A45]
	(No symbol) [0x00007FF7E1725AD4]
	GetHandleVerifier [0x00007FF7E1C4D5BB+3695675]
	GetHandleVerifier [0x00007FF7E1CA6197+4059159]
	GetHandleVerifier [0x00007FF7E1C9DF63+4025827]
	GetHandleVerifier [0x00007FF7E196F029+687785]
	(No symbol) [0x00007FF7E184B508]
	(No symbol) [0x00007FF7E1847564]
	(No 

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv

# Function to fetch additional information from a given URL
def fetch_additional_info(url):
    # Set up the WebDriver
    driver = webdriver.Chrome()
    driver.get(url)

    try:
        # Wait for the close button to be clickable
        close_popup_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'UpdateCityModal__cancel-btn___2jWwS'))
        )

        # Click the close button
        close_popup_button.click()
    except Exception as e:
        print(f"Error while handling popup: {e}")

    # Extract information using BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Extracting the Name, Brand Name, Rating, Number of Rating, Key Benefits, Key Ingredients
    name_of_medicine_element = soup.find('h1', {'class': 'ProductTitle__product-title___3QMYH'})
    name_of_medicine = name_of_medicine_element.text.strip() if name_of_medicine_element else 'NA'

    brand_name_element = soup.find('div', {'class': 'ProductTitle__marketer___7Wsj9'})
    brand_name = brand_name_element.text.strip() if brand_name_element else 'NA'

    rating_element = soup.find('div', {'class': 'RatingDisplay__ratings-container___3oUuo'})
    rating = rating_element.text.strip() if rating_element else 'NA'

    num_rating_element = soup.find('span', {'class': 'RatingDisplay__ratings-header___ZNj5b'})
    num_rating = num_rating_element.text.strip() if num_rating_element else 'NA'

    key_benefits_element = soup.find('div', {'class': 'ProductHighlights__product-highlights___2jAF5'})
    key_benefits_items = key_benefits_element.find_all('li') if key_benefits_element else []
    key_benefits_data = [item.text.strip() for item in key_benefits_items] if key_benefits_items else 'NA'
    key_benefits = '\n'.join(f'{index + 1}. {element}' for index, element in enumerate(key_benefits_data, start=0)) if key_benefits_data != 'NA' else 'NA'

    # Close the WebDriver
    driver.quit()

    return name_of_medicine, brand_name, rating, num_rating, key_benefits

# Specify the path to your CSV files
csv_file_path_input = 'medicine_name_data_1200.csv'
csv_file_path_output = 'medicine_details.csv'

# Create a list to store all the 1Mg URLs
all_1mg_urls = []

# Read data from the CSV file and extract '1Mg_URL'
with open(csv_file_path_input, 'r', encoding='utf-8') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        url = row['1Mg_URL']
        all_1mg_urls.append(url)

# Create a list to store the fetched information
all_additional_info = []

# Iterate through each URL and fetch additional information
for url in all_1mg_urls:
    name, brand, rating, num_rating, key_benefits = fetch_additional_info(url)
    all_additional_info.append({
        'Name': name,
        'Brand_Name': brand,
        'Rating': rating,
        'Number_of_rating': num_rating,
        'Key Benefits': key_benefits
    })

# Create a new CSV file and write all the data
with open(csv_file_path_output, 'w', newline='', encoding='utf-8') as csvfile:
    # Create a CSV writer
    csv_writer = csv.writer(csvfile)

    # Write header row
    header = ['Name', 'Brand_Name', 'Rating', 'Number_of_rating', 'Key Benefits']
    csv_writer.writerow(header)

    # Write all the data to the CSV file
    for info in all_additional_info:
        csv_writer.writerow([
            info['Name'],
            info['Brand_Name'],
            info['Rating'],
            info['Number_of_rating'],
            info['Key Benefits']
        ])


Error while handling popup: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E18D4D02+56194]
	(No symbol) [0x00007FF7E18404B2]
	(No symbol) [0x00007FF7E16E76AA]
	(No symbol) [0x00007FF7E17316D0]
	(No symbol) [0x00007FF7E17317EC]
	(No symbol) [0x00007FF7E1774D77]
	(No symbol) [0x00007FF7E1755EBF]
	(No symbol) [0x00007FF7E1772786]
	(No symbol) [0x00007FF7E1755C23]
	(No symbol) [0x00007FF7E1724A45]
	(No symbol) [0x00007FF7E1725AD4]
	GetHandleVerifier [0x00007FF7E1C4D5BB+3695675]
	GetHandleVerifier [0x00007FF7E1CA6197+4059159]
	GetHandleVerifier [0x00007FF7E1C9DF63+4025827]
	GetHandleVerifier [0x00007FF7E196F029+687785]
	(No symbol) [0x00007FF7E184B508]
	(No symbol) [0x00007FF7E1847564]
	(No symbol) [0x00007FF7E18476E9]
	(No symbol) [0x00007FF7E1838094]
	BaseThreadInitThunk [0x00007FFDF9E7257D+29]
	RtlUserThreadStart [0x00007FFDFA9EAA58+40]

Error while handling popup: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E18D4D02+56194]
	(No symbol) [0x00007FF7E18404B2]
	(No symbol) [0x0

TimeoutException: Message: timeout: Timed out receiving message from renderer: 298.922
  (Session info: chrome=120.0.6099.217)
Stacktrace:
	GetHandleVerifier [0x00007FF7E18D4D02+56194]
	(No symbol) [0x00007FF7E18404B2]
	(No symbol) [0x00007FF7E16E76AA]
	(No symbol) [0x00007FF7E16D0839]
	(No symbol) [0x00007FF7E16D06EB]
	(No symbol) [0x00007FF7E16CEE3D]
	(No symbol) [0x00007FF7E16CF7CE]
	(No symbol) [0x00007FF7E16DD6CD]
	(No symbol) [0x00007FF7E16F4471]
	(No symbol) [0x00007FF7E16F937A]
	(No symbol) [0x00007FF7E16CFEC6]
	(No symbol) [0x00007FF7E16F41CF]
	(No symbol) [0x00007FF7E177297F]
	(No symbol) [0x00007FF7E1755C23]
	(No symbol) [0x00007FF7E1724A45]
	(No symbol) [0x00007FF7E1725AD4]
	GetHandleVerifier [0x00007FF7E1C4D5BB+3695675]
	GetHandleVerifier [0x00007FF7E1CA6197+4059159]
	GetHandleVerifier [0x00007FF7E1C9DF63+4025827]
	GetHandleVerifier [0x00007FF7E196F029+687785]
	(No symbol) [0x00007FF7E184B508]
	(No symbol) [0x00007FF7E1847564]
	(No symbol) [0x00007FF7E18476E9]
	(No symbol)

In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
import time
from selenium.webdriver.common.by import By

# Function to fetch additional information from a given URL
def fetch_additional_info(url):
    driver = webdriver.Chrome()
    driver.get(url)
    
    # Wait for some time to ensure the page is fully loaded
    driver.implicitly_wait(2)
    
    # Extract information using BeautifulSoup 
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Remove dropdown
    close_popup_button = driver.find_element(By.CLASS_NAME, 'UpdateCityModal__cancel-btn___2jWwS')
    close_popup_button.click()
    time.sleep(2)

    # Find the main section
    main = soup.find('div', {'class': 'ProductDescription__description-content___A_qCZ'})
    
   
    
    # Find all ul elements within the specific div class
    ul_elements = main.find_all('ul')
    
    # Extract text content of all li elements within each ul
    ul_data = []
    for ul in ul_elements:
        li_texts = [li.text.strip() for li in ul.find_all('li')]
        ul_data.append(li_texts)
    
    # Close the WebDriver
    driver.quit()
    
    return ul_data

# Specify the path to your CSV file
csv_file_path = 'new_medicines_data_28.csv'

# Create a list to store all the 1Mg URLs
all_1mg_urls = []

# Read data from the CSV file and extract '1Mg_URL'
with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        url = row['1Mg_URL']
        all_1mg_urls.append(url)

# Create a list to store the fetched information
all_additional_info = []

# Iterate through each URL and fetch additional information
for url in all_1mg_urls:
    ul_data = fetch_additional_info(url)
    all_additional_info.append({
        'UL_Data': ul_data
    })

# Print the fetched information
for info in all_additional_info:
    print(info)


{'UL_Data': [['It helps improve mood and energy levels'], ['Relieves itchy scalp and severe headache accompanied by acidity and nausea'], ['Helps reduce eruptions of pimples on the face'], ['Soothes itchy, burning eyes with blurred vision and clears redness of the eyes'], ['Improves digestion and defecation'], ['Relieves burning sensation in the urine or decreased urine frequency'], ['Reduces stiffness in legs and soothes the pain'], ['Helpful in alleviating skin issues like dry eczema, psoriasis, and glandular swellings'], ['Read the label carefully before use'], ['Do not exceed the recommended dose'], ['Keep out of the reach of children'], ['Store in a cool and dry place, away from direct sunlight and heat'], ['Avoid any strong smell in the mouth, such as coffee, onion, hing, mint, camphor, garlic etc., while taking the medicine'], ['Keep at least half an hour gap between food/drink/ any other medicines and allopathic medicine']]}
{'UL_Data': [['Blatta Orientalis'], ['Bryonia Alba'],